In [5]:

! pip install transformers gradio -q

     |████████████████████████████████| 11.6 MB 4.1 MB/s 
     |████████████████████████████████| 2.3 MB 15.9 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 278 kB 67.9 MB/s 
     |████████████████████████████████| 106 kB 71.7 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 213 kB 67.6 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 80 kB 10.2 MB/s 
     |████████████████████████████████| 68 kB 7.9 MB/s 
     |████████████████████████████████| 68 kB 7.5 MB/s 
     |████████████████████████████████| 68 kB 7.3 MB/s 
     |████████████████████████████████| 68 kB 7.7 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |███████████████████████████████

In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import gradio as gr

In [7]:
# Instantiate model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

In [113]:
def predict(input, history=[]):

    instruction = 'Instruction: given a dialog context, you need to response empathically'

    knowledge = '  '

    s = list(sum(history, ()))

    s.append(input)

    #print(s)

    dialog = ' EOS ' .join(s)

    #print(dialog)

    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"

    top_p = 0.9
    min_length = 8
    max_length = 200


    # tokenize the new input sentence
    new_user_input_ids = tokenizer.encode(f"{query}", return_tensors='pt')


    output = model.generate(new_user_input_ids, 
                            min_length=int(min_length), 
                            max_length=int(max_length), 
                            top_p=top_p, 
                            do_sample=True).tolist()
    
  
    response = tokenizer.decode(output[0], skip_special_tokens=True)

 
    history.append((input, response))
    
    return history

In [198]:
def predict2(input, history=[]):

    # Basic instruction
    instruction = 'Instruction: given a dialog context, you need to response empathically.'

    # Knowledge given
    knowledge = '  '

    # Copy history of dialog
    dialog_list = history.copy()
    print('history of dialog: ', dialog_list)

    # Append user input to dialog
    dialog_list.append(input)
    print('new input appended: ', dialog_list)

    # Prepare dialog to be used in the model
    dialog = ' EOS ' .join(dialog_list)
    print('dialog prepared for the model: ', dialog)

    # Build query
    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"
    print('query: ', query)

    # Tokenize the new input sentence
    new_user_input_ids = tokenizer.encode(f"{query}", return_tensors='pt')
    print('Output of the tokenizer: ', new_user_input_ids)

    # Set params for model output
    top_p = 0.9 # What is this???
    min_length = 8
    max_length = 200

    # Generate output
    output = model.generate(new_user_input_ids, 
                            min_length=int(min_length), 
                            max_length=int(max_length), 
                            top_p=top_p, 
                            do_sample=True).tolist()
    print('Output of the model: ', output)          

    # Decode output
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print('Decoded output: ', response)     

    # Create new history of dialog
    dialog_list.append(response)

    return response, dialog_list

In [199]:
# First user input
input1 = 'I am sad.'

In [205]:
# First response
response1, history1 = predict2(input1)
print(response1)
print(history1)

history of dialog:  []
new input appended:  ['I am sad.']
dialog prepared for the model:  I am sad.
query:  Instruction: given a dialog context, you need to response empathically. [CONTEXT] I am sad.   
Output of the tokenizer:  tensor([[21035,    10,   787,     3,     9, 13463,  2625,     6,    25,   174,
            12,  1773,  8943,     9,   189,  6402,     5,   784, 17752,  3463,
             4,   382,   908,    27,   183,  6819,     5,     1]])
Output of the model:  [[0, 27, 183, 78, 8032, 24, 25, 33, 352, 190, 48, 5, 1]]
Decoded output:  I am so sorry that you are going through this.
I am so sorry that you are going through this.
['I am sad.', 'I am so sorry that you are going through this.']


In [206]:
# Second user input
input2 = 'Can you help me?'


In [213]:
# Second response
response2, history2 = predict2(input2, history1)
print(response2)
print(history2)

history of dialog:  ['I am sad.', 'I am so sorry that you are going through this.']
new input appended:  ['I am sad.', 'I am so sorry that you are going through this.', 'Can you help me?']
dialog prepared for the model:  I am sad. EOS I am so sorry that you are going through this. EOS Can you help me?
query:  Instruction: given a dialog context, you need to response empathically. [CONTEXT] I am sad. EOS I am so sorry that you are going through this. EOS Can you help me?   
Output of the tokenizer:  tensor([[21035,    10,   787,     3,     9, 13463,  2625,     6,    25,   174,
            12,  1773,  8943,     9,   189,  6402,     5,   784, 17752,  3463,
             4,   382,   908,    27,   183,  6819,     5, 32100,    27,   183,
            78,  8032,    24,    25,    33,   352,   190,    48,     5, 32100,
          1072,    25,   199,   140,    58,     1]])
Output of the model:  [[0, 863, 752, 140, 214, 24, 27, 183, 352, 190, 48, 5, 1]]
Decoded output:  Please let me know that I am 

In [214]:
# Third user input
input3 = 'I do not manage to deal with it alone...'

In [215]:
# Third response
response3, history3 = predict2(input3, history2)
print(response3)
print(history3)

history of dialog:  ['I am sad.', 'I am so sorry that you are going through this.', 'Can you help me?', 'Please let me know that I am going through this.']
new input appended:  ['I am sad.', 'I am so sorry that you are going through this.', 'Can you help me?', 'Please let me know that I am going through this.', 'I do not manage to deal with it alone...']
dialog prepared for the model:  I am sad. EOS I am so sorry that you are going through this. EOS Can you help me? EOS Please let me know that I am going through this. EOS I do not manage to deal with it alone...
query:  Instruction: given a dialog context, you need to response empathically. [CONTEXT] I am sad. EOS I am so sorry that you are going through this. EOS Can you help me? EOS Please let me know that I am going through this. EOS I do not manage to deal with it alone...   
Output of the tokenizer:  tensor([[21035,    10,   787,     3,     9, 13463,  2625,     6,    25,   174,
            12,  1773,  8943,     9,   189,  6402,   